In [51]:
import os
import json
import re
import csv

In [52]:
def remove_emojis_and_links(text):
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = text.replace("\n", "")
    return text.strip()

In [53]:
dem_accs = os.listdir("dem")
rep_accs = os.listdir("rep")

In [54]:
tweets = []
for i in range(len(dem_accs)):
    dem_tweets = os.listdir(os.path.join("dem", dem_accs[i]))
    for j in range(len(dem_tweets)):
        data = json.load(open(os.path.join("dem", dem_accs[i], dem_tweets[j])))
        tweet_text = remove_emojis_and_links(data["rawContent"])
        if tweet_text != "":
            tweets.append(tweet_text)

In [57]:
with open("democrats.csv", 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["text"])
    for tweet in tweets:
        writer.writerow([tweet])

In [58]:
tweets = []
for i in range(len(rep_accs)):
    rep_tweets = os.listdir(os.path.join("rep", rep_accs[i]))
    for j in range(len(rep_tweets)):
        data = json.load(open(os.path.join("rep", rep_accs[i], rep_tweets[j])))
        tweet_text = remove_emojis_and_links(data["rawContent"])
        if tweet_text != "":
            tweets.append(tweet_text)

In [59]:
with open("republicans.csv", 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["text"])
    for tweet in tweets:
        writer.writerow([tweet])